Follow along from : https://www.analyticsvidhya.com/blog/2016/09/comprehensive-introduction-to-apache-spark-rdds-dataframes-using-pyspark/

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf()
conf.setAppName('mySparkApp')
conf.setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)


In [2]:
nums = sc.parallelize([1,2,3,4])
nums.count()

4

In [3]:
data = range(1,1000)
rdd = sc.parallelize(data)
rdd.collect()
rdd.take(2) # It will print first 2 elements of rdd

[1, 2]

In [4]:
data = ['Hello' , 'I' , 'AM', 'Ankit ', 'Gupta']
Rdd = sc.parallelize(data)

In [5]:
Rdd1 = Rdd.map(lambda x: (x,1))
Rdd1.collect()

[('Hello', 1), ('I', 1), ('AM', 1), ('Ankit ', 1), ('Gupta', 1)]

In [9]:
# I think I can just use the SparkSession created above instead of SQLContext
#from pyspark import SQLContext
#sqlContext = SQLContext(sc)

#C:\Users\Dave\anaconda3\envs\pyspark\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
#  warnings.warn(

C:\Users\Dave\anaconda3\envs\pyspark\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
#train = sqlContext.load(source="com.databricks.spark.csv", path = 'PATH/train.csv', header = True,inferSchema = True)
#test = sqlContext.load(source="com.databricks.spark.csv", path = 'PATH/test-comb.csv', header = True,inferSchema = True)
train = spark.read.csv('data/train.csv', header = True,inferSchema = True)
test = spark.read.csv('data/test.csv', header = True,inferSchema = True)

In [8]:
train.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [9]:
train.head(10)

[Row(User_ID=1000001, Product_ID='P00069042', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=3, Product_Category_2=None, Product_Category_3=None, Purchase=8370),
 Row(User_ID=1000001, Product_ID='P00248942', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=1, Product_Category_2=6, Product_Category_3=14, Purchase=15200),
 Row(User_ID=1000001, Product_ID='P00087842', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=None, Product_Category_3=None, Purchase=1422),
 Row(User_ID=1000001, Product_ID='P00085442', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=14, Product_Category_3=None, Purchase=1057),
 Row(User_ID=1000002, Product_ID='P0

In [10]:
train.count()

550068

Impute Missing values

We can check number of not null observations in train and test by calling drop() method. By default, drop() method will drop a row if it contains any null value. We can also pass ‘all” to drop a row only if all its values are null.

In [11]:
train.na.drop().count(),test.na.drop('any').count()

(166821, 71037)

Here, I am imputing null values in train and test file with -1. Imputing the values with -1 is not an elegant solution. We have several algorithms / techniques to impute null values, but for the simplicity I am imputing null with constant value (-1). We can transform our base train, test Dataframes after applying this imputation. For imputing constant value, we have fillna method. Lets fill the -1 in-place of null in all columns.

In [12]:
train = train.fillna(-1)
test = test.fillna(-1)

Analyzing numerical features

We can also see the various summary Statistics of a Dataframe columns using describe() method, which shows statistics for numerical variables. To show the results we need to call show() method.

In [13]:
train.describe().show()

+-------+------------------+----------+------+------+------------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|        Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+------------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|            550068|       550068|                    550068|             550068|            550068|            550068|            550068|           550068|
|   mean|1003028.8424013031|      null|  null|  null| 8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 6.419769919

Sub-setting Columns

Let’s select a column called ‘User_ID’ from a train, we need to call a method ‘select’ and pass the column name which we want to select. The select method will show result for selected column. We can also select more than one column from a data frame by providing columns name separated by comma.

In [14]:
train.select('User_ID').show()

+-------+
|User_ID|
+-------+
|1000001|
|1000001|
|1000001|
|1000001|
|1000002|
|1000003|
|1000004|
|1000004|
|1000004|
|1000005|
|1000005|
|1000005|
|1000005|
|1000005|
|1000006|
|1000006|
|1000006|
|1000006|
|1000007|
|1000008|
+-------+
only showing top 20 rows



Analyzing categorical features

To start building a model, we need to see the distribution of categorical features in train and test. Here I am showing this for only Product_ID but we can also do the same for any categorical feature. Let’s see the number of distinct categories of “Product_ID” in train and test. Which we can do by applying methods distinct() and count().

In [15]:
train.select('Product_ID').distinct().count(), test.select('Product_ID').distinct().count()

(3631, 3491)

After counting the number of distinct values for train and test we can see the train has more categories than test. Let us check what are the categories for Product_ID, which are in test but not in train by applying subtract method.We can also do the same for all categorical feature.

In [16]:
diff_cat_in_train_test=test.select('Product_ID').subtract(train.select('Product_ID'))
diff_cat_in_train_test.distinct().count()# For distict count

46

Above you can see that 46 different categories are in test not in train. In this case, either we collect more data about them or skip the rows in test for those categories(invalid category) which are not in train.

Transforming categorical variables to labels

We also need to transform categorical columns to label by applying StringIndexer Transformation on Product_ID which will encode the Product_ID column of labels to a column of label indices. You can see more about this from the link

In [17]:
from pyspark.ml.feature import StringIndexer
plan_indexer = StringIndexer(inputCol = 'Product_ID', outputCol = 'product_ID')
labeller = plan_indexer.fit(train)

Above, we build a ‘labeller’ by applying fit() method on train Dataframe. Later we will use this ‘labeller’ to transform our train and test. Let us transform our train and test Dataframe with the help of labeller. We need to call transform method for doing that. We will store the transformation result in Train1 and Test1.

In [18]:
Train1 = labeller.transform(train)
Test1 = labeller.transform(test)

In [19]:
Train1.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001|     765.0|     F| 0-17|        10|            A|                         2|             0|                 3|                -1|                -1|    8370|
|1000001|     183.0|     F| 0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001|    1496.0|     F| 0-17|        10|            A|                         2|             0|                12|                -1|                -1|    1422

Selecting Features to Build a Machine Learning Model

Let’s try to create a formula for Machine learning model like we do in R. First, we need to import RFormula from the pyspark.ml.feature. Then we need to specify the dependent and independent column inside this formula. We also have to specify the names for features column and label column.

In [20]:
from pyspark.ml.feature import RFormula
formula = RFormula(formula="Purchase ~ Age+ Occupation +City_Category+Stay_In_Current_City_Years+Product_Category_1+Product_Category_2+ Gender",featuresCol="features",labelCol="label")

After creating the formula we need to fit this formula on our Train1 and transform Train1,Test1 through this formula. Lets see how to do this and after fitting transform train1,Test1 in train1,test1.

In [21]:
t1 = formula.fit(Train1)
train1 = t1.transform(Train1)
test1 = t1.transform(Test1)

In [22]:
train1.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------------+-------+
|User_ID|product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|            features|  label|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------------+-------+
|1000001|     765.0|     F| 0-17|        10|            A|                         2|             0|                 3|                -1|                -1|    8370|(16,[6,10,13,14],...| 8370.0|
|1000001|     183.0|     F| 0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|(16,[6,10,13,14],...|15200.0|
|1000001|    1496.0|

After applying the formula we can see that train1 and test1 have 2 extra columns called features and label those we have specified in the formula (featuresCol=”features” and labelCol=”label”). The intuition is that all categorical variables in the features column in train1 and test1 are transformed to the numerical and the numerical variables are same as before for applying ML. Purchase variable will transom to label column. We can also look at the column features and label in train1 and test1.

In [23]:
train1.select('features').show()
train1.select('label').show()

+--------------------+
|            features|
+--------------------+
|(16,[6,10,13,14],...|
|(16,[6,10,13,14],...|
|(16,[6,10,13,14],...|
|(16,[6,10,13,14],...|
|(16,[5,6,8,12,13,...|
|(16,[0,6,11,13,14...|
|(16,[3,6,7,10,13,...|
|(16,[3,6,7,10,13,...|
|(16,[3,6,7,10,13,...|
|(16,[0,6,9,13,14,...|
|(16,[0,6,9,13,14,...|
|(16,[0,6,9,13,14,...|
|(16,[0,6,9,13,14,...|
|(16,[0,6,9,13,14,...|
|(16,[4,6,9,13,14]...|
|(16,[4,6,9,13,14]...|
|(16,[4,6,9,13,14]...|
|(16,[4,6,9,13,14]...|
|(16,[1,6,7,9,13,1...|
|(16,[0,6,8,12,13,...|
+--------------------+
only showing top 20 rows

+-------+
|  label|
+-------+
| 8370.0|
|15200.0|
| 1422.0|
| 1057.0|
| 7969.0|
|15227.0|
|19215.0|
|15854.0|
|15686.0|
| 7871.0|
| 5254.0|
| 3957.0|
| 6073.0|
|15665.0|
| 5378.0|
| 2079.0|
|13055.0|
| 8851.0|
|11788.0|
|19614.0|
+-------+
only showing top 20 rows



Building a Machine Learning Model: Random Forest

After applying the RFormula and transforming the Dataframe, we now need to develop the machine learning model on this data. I want to apply a random forest regressor for this task. Let us import a random forest regressor, which is defined in pyspark.ml.regression and then create a model called rf. I am going to use default parameters for randomforest algorithm.

In [24]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()

After creating a model rf we need to divide our train1 data to train_cv and test_cv for cross validation.

Here we are dividing train1 Dataframe in 70% for train_cv and 30% test_cv.

In [25]:
(train_cv, test_cv) = train1.randomSplit([0.7, 0.3])

In [26]:
# Now build the model on train_cv and predict on test_cv. The results will save in  predictions.
model1 = rf.fit(train_cv)
predictions = model1.transform(test_cv)

In [28]:
# If you check the columns in predictions Dataframe, there is one column called prediction which has prediction result for test_cv.
predictions.show()

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------------+-------+------------------+
|User_ID|product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|            features|  label|        prediction|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------------+-------+------------------+
|1000001|       1.0|     F|0-17|        10|            A|                         2|             0|                 1|                 2|                 9|   15416|(16,[6,10,13,14],...|15416.0|13106.263531426179|
|1000001|      16.0|     F|0-17|        10|            A|                         2|             0|                 1|                 2|       

Lets evaluate our predictions on test_cv and see what is the mean squae error.

To evaluate model we need to import RegressionEvaluator from the pyspark.ml.evaluation. We have to create an object for this. There is a method called evaluate for evaluator which will evaluate the model. We need to specify the metrics for that.

In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()
mse = evaluator.evaluate(predictions,{evaluator.metricName:"mse" })
import numpy as np
np.sqrt(mse), mse

(3871.8447318281524, 14991181.627385417)

After evaluation we can see that our root mean square error is 3773.1460883883865 which is a square root of mse.

Now, we will implement the same process on full train1 dataset.

In [35]:
model = rf.fit(train1)
predictions1 = model.transform(test1)

In [39]:
# After prediction, we need to select those columns which are required in Black Friday competition submission.
df = predictions1.selectExpr("User_ID as User_ID", "Product_ID as Product_ID", "prediction as Purchase")

In [40]:
# Now we need to write the df in csv format for submission.
df.show()
# Not making the jump to python
df.toPandas().show()

#df.toPandas().to_csv('submission.csv')


+-------+----------+------------------+
|User_ID|Product_ID|          Purchase|
+-------+----------+------------------+
|1000004|      47.0|13343.564238703912|
|1000009|     683.0| 7571.386861104364|
|1000010|    1188.0| 6810.857837067696|
|1000010|    2866.0| 6810.857837067696|
|1000011|     193.0| 6879.479569910332|
|1000013|     574.0| 11248.97582695539|
|1000013|     334.0|12797.228817566061|
|1000013|    1008.0| 11248.97582695539|
|1000015|    1305.0| 8084.808476040179|
|1000022|     420.0| 6891.603473330273|
|1000026|       6.0|13295.424160294784|
|1000026|    1154.0| 6942.365281084855|
|1000026|     426.0| 6725.995605055779|
|1000026|     606.0| 6725.995605055779|
|1000028|     262.0| 8156.773606331614|
|1000029|     312.0|   12685.222152834|
|1000033|     885.0| 9280.889205648358|
|1000033|     598.0| 7126.924556223976|
|1000034|       0.0| 6952.060731188198|
|1000035|    1009.0| 11248.97582695539|
+-------+----------+------------------+
only showing top 20 rows



Py4JJavaError: An error occurred while calling o753.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 126.0 failed 1 times, most recent failure: Lost task 0.0 in stage 126.0 (TID 108) (DESKTOP-8RP3DQ1 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$3703/1356671337: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at org.apache.spark.